# Ollama + OpenAI + Python

## 1. Eleza jina la modeli

Ikiwa ulileta modeli tofauti na "phi3:mini", badilisha thamani katika seli hapa chini. Kigezo hicho kitatumika katika msimbo kote kwenye daftari.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Kuseti mteja wa Open AI

Kwa kawaida, mteja wa OpenAI hutumika na OpenAI.com au Azure OpenAI kuwasiliana na mifano mikubwa ya lugha.  
Hata hivyo, inaweza pia kutumika na Ollama, kwa kuwa Ollama inatoa sehemu ya mwisho inayolingana na OpenAI kwenye "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Tengeneza majibu ya mazungumzo

Sasa tunaweza kutumia OpenAI SDK kuunda majibu kwa mazungumzo. Ombi hili linapaswa kutengeneza haiku kuhusu paka:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Uhandisi wa maelekezo

Ujumbe wa kwanza unaotumwa kwa modeli ya lugha unaitwa "ujumbe wa mfumo" au "maelekezo ya mfumo", na unatoa maagizo ya jumla kwa modeli.
Unaweza kutoa maelekezo yako ya mfumo maalum ili kuelekeza modeli ya lugha kutoa matokeo kwa njia tofauti.
Badilisha `SYSTEM_MESSAGE` hapa chini ili kujibu kama mhusika maarufu wa filamu/TV unayempenda, au pata msukumo wa maelekezo mengine ya mfumo kutoka [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Baada ya kubadilisha ujumbe wa mfumo, toa swali la kwanza la mtumiaji katika `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Mifano ya maswali machache

Njia nyingine ya kuelekeza modeli ya lugha ni kutoa "maswali machache", mfululizo wa mifano ya maswali/majibu yanayoonyesha jinsi inavyopaswa kujibu.

Mfano hapa chini unajaribu kufanya modeli ya lugha ifanye kazi kama msaidizi wa kufundisha kwa kutoa mifano michache ya maswali na majibu ambayo msaidizi wa kufundisha anaweza kutoa, kisha inatoa swali ambalo mwanafunzi anaweza kuuliza.

Jaribu kwanza, kisha badilisha `SYSTEM_MESSAGE`, `EXAMPLES`, na `USER_MESSAGE` kwa hali mpya.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Uzalishaji Ulioimarishwa kwa Urejeshaji

RAG (Uzalishaji Ulioimarishwa kwa Urejeshaji) ni mbinu ya kupata modeli ya lugha kujibu maswali kwa usahihi katika uwanja fulani, kwa kuanza kwa kurejesha taarifa husika kutoka kwa chanzo cha maarifa na kisha kutoa jibu kulingana na taarifa hiyo.

Tumetoa faili ya ndani ya CSV yenye data kuhusu magari ya mseto. Nambari iliyo hapa chini inasoma faili ya CSV, inatafuta mechi za swali la mtumiaji, na kisha inazalisha jibu kulingana na taarifa iliyopatikana. Kumbuka kuwa hii itachukua muda mrefu zaidi kuliko mifano ya awali, kwa kuwa inatuma data zaidi kwa modeli. Ikiwa utaona jibu bado halijazingatia data, unaweza kujaribu uhandisi wa mfumo au kujaribu modeli nyingine. Kwa ujumla, RAG ni bora zaidi na modeli kubwa au na matoleo yaliyorekebishwa ya SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
